In [1]:
import timeit
import pandas as pd
import polars as pl

### Load data

In [2]:
# pandas
start = timeit.default_timer()
pd_df = pd.read_csv("./data_28.csv")
stop = timeit.default_timer()
print('Time: ', stop - start)  

Time:  56.8925461


In [2]:
# polars
start = timeit.default_timer()
pl_df = pl.scan_csv("./data_28.csv")
pl_df = pl_df.collect()
stop = timeit.default_timer()
print('Time: ', stop - start)  

Time:  12.313136199999999


### Filter groupby agg

In [3]:
# pandas
start = timeit.default_timer()

pd_df_agg = pd_df[pd_df["demand"] > 0]\
.groupby("group_id")['demand']\
.sum()

print(pd_df_agg)

stop = timeit.default_timer()
print('Time: ', stop - start)  

group_id
0     1.433424e+08
1     2.584172e+07
2     7.516780e+05
3     4.403254e+07
4     7.253725e+06
5     6.477589e+06
6     4.058315e+07
7     1.489211e+08
8     1.879766e+07
9     5.338499e+07
10    3.436026e+06
11    4.322650e+07
12    2.555600e+05
13    3.740526e+06
14    5.847329e+06
Name: demand, dtype: float64
Time:  22.266677900000005


In [3]:
# polars
start = timeit.default_timer()

pl_df_agg = (
    pl_df
    .filter(pl.col("demand") > 0)
    .groupby("group_id")
    .agg(pl.col("demand").sum())
)

print(pl_df_agg)

stop = timeit.default_timer()
print('Time: ', stop - start)  

shape: (15, 2)
┌──────────┬──────────┐
│ group_id ┆ demand   │
│ ---      ┆ ---      │
│ i64      ┆ f64      │
╞══════════╪══════════╡
│ 0        ┆ 1.4334e8 │
│ 12       ┆ 255560.0 │
│ 5        ┆ 6.4776e6 │
│ 7        ┆ 1.4892e8 │
│ …        ┆ …        │
│ 4        ┆ 7.2537e6 │
│ 13       ┆ 3.7405e6 │
│ 3        ┆ 4.4033e7 │
│ 9        ┆ 5.3385e7 │
└──────────┴──────────┘
Time:  0.9004170000000009


## Calculate rolling mean

In [4]:
# pandas
start = timeit.default_timer()


pd_df["rolling_mean"] = pd_df.groupby(['product_id', 'store_id'])['demand']\
                            .transform(lambda s: s.rolling(10, min_periods=1).mean())

print(pd_df)

stop = timeit.default_timer()
print('Time: ', stop - start)  

          week  product_id  store_id  group_id    demand  s_group_id     tag  \
0          576           0       244         0  5.000000           1  before   
1          577           0       244         0  5.000000           1  before   
2          578           0       244         0  5.000000           1  before   
3          579           0       244         0  5.000000           1  before   
4          580           0       244         0  5.000000           1  before   
...        ...         ...       ...       ...       ...         ...     ...   
91554683  1039         764       395        14  5.357839           2   after   
91554684  1040         764       395        14  5.187080           2   after   
91554685  1041         764       395        14  7.220590           2   after   
91554686  1042         764       395        14  6.403280           2   after   
91554687  1043         764       395        14  5.376021           2   after   

          rolling_mean  
0             

In [4]:
# polars
start = timeit.default_timer()

pl_df = (
    pl_df.select(
    [
        "week",
        "product_id",
        "store_id",
        "group_id",
        "demand",
        "s_group_id",
        "tag",
        pl.col("demand")\
        .rolling_mean(10, min_periods=1)\
        .over('product_id', 'store_id')\
        .alias("rolling_mean")
    ]
    )
)

print(pl_df)

stop = timeit.default_timer()
print('Time: ', stop - start) 

shape: (91_554_688, 8)
┌──────┬────────────┬──────────┬──────────┬──────────┬────────────┬────────┬──────────────┐
│ week ┆ product_id ┆ store_id ┆ group_id ┆ demand   ┆ s_group_id ┆ tag    ┆ rolling_mean │
│ ---  ┆ ---        ┆ ---      ┆ ---      ┆ ---      ┆ ---        ┆ ---    ┆ ---          │
│ i64  ┆ i64        ┆ i64      ┆ i64      ┆ f64      ┆ i64        ┆ str    ┆ f64          │
╞══════╪════════════╪══════════╪══════════╪══════════╪════════════╪════════╪══════════════╡
│ 576  ┆ 0          ┆ 244      ┆ 0        ┆ 5.0      ┆ 1          ┆ before ┆ 5.0          │
│ 577  ┆ 0          ┆ 244      ┆ 0        ┆ 5.0      ┆ 1          ┆ before ┆ 5.0          │
│ 578  ┆ 0          ┆ 244      ┆ 0        ┆ 5.0      ┆ 1          ┆ before ┆ 5.0          │
│ 579  ┆ 0          ┆ 244      ┆ 0        ┆ 5.0      ┆ 1          ┆ before ┆ 5.0          │
│ …    ┆ …          ┆ …        ┆ …        ┆ …        ┆ …          ┆ …      ┆ …            │
│ 1040 ┆ 764        ┆ 395      ┆ 14       ┆ 5.18708  ┆ 2 

## Join

In [5]:
# pandas
start = timeit.default_timer()

pd_df_agg = pd_df[pd_df["demand"] > 0]\
.groupby("group_id")['demand']\
.sum()

pd_df = pd_df.merge(pd_df_agg,
                   on="group_id")

print(pd_df)

stop = timeit.default_timer()
print('Time: ', stop - start)  

          week  product_id  store_id  group_id  demand_x  s_group_id     tag  \
0          576           0       244         0  5.000000           1  before   
1          577           0       244         0  5.000000           1  before   
2          578           0       244         0  5.000000           1  before   
3          579           0       244         0  5.000000           1  before   
4          580           0       244         0  5.000000           1  before   
...        ...         ...       ...       ...       ...         ...     ...   
91554683  1039         764       395        14  5.357839           2   after   
91554684  1040         764       395        14  5.187080           2   after   
91554685  1041         764       395        14  7.220590           2   after   
91554686  1042         764       395        14  6.403280           2   after   
91554687  1043         764       395        14  5.376021           2   after   

          rolling_mean      demand_y  


In [5]:
# polars
start = timeit.default_timer()

pl_df_agg = (
    pl_df
    .filter(pl.col("demand") > 0)
    .groupby("group_id")
    .agg(pl.col("demand").sum())
)

pl_df = pl_df.join(pl_df_agg, on="group_id")

print(pl_df)

stop = timeit.default_timer()
print('Time: ', stop - start)  

shape: (91_554_688, 9)
┌──────┬────────────┬──────────┬──────────┬───┬────────────┬────────┬──────────────┬──────────────┐
│ week ┆ product_id ┆ store_id ┆ group_id ┆ … ┆ s_group_id ┆ tag    ┆ rolling_mean ┆ demand_right │
│ ---  ┆ ---        ┆ ---      ┆ ---      ┆   ┆ ---        ┆ ---    ┆ ---          ┆ ---          │
│ i64  ┆ i64        ┆ i64      ┆ i64      ┆   ┆ i64        ┆ str    ┆ f64          ┆ f64          │
╞══════╪════════════╪══════════╪══════════╪═══╪════════════╪════════╪══════════════╪══════════════╡
│ 576  ┆ 0          ┆ 244      ┆ 0        ┆ … ┆ 1          ┆ before ┆ 5.0          ┆ 1.4334e8     │
│ 577  ┆ 0          ┆ 244      ┆ 0        ┆ … ┆ 1          ┆ before ┆ 5.0          ┆ 1.4334e8     │
│ 578  ┆ 0          ┆ 244      ┆ 0        ┆ … ┆ 1          ┆ before ┆ 5.0          ┆ 1.4334e8     │
│ 579  ┆ 0          ┆ 244      ┆ 0        ┆ … ┆ 1          ┆ before ┆ 5.0          ┆ 1.4334e8     │
│ …    ┆ …          ┆ …        ┆ …        ┆ … ┆ …          ┆ …      ┆ …      